In [ ]:
# InterpolationCodeUsage.ipynb testing

In [2]:
from utility_programs.read_routines import SAMI

from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator, griddata
import numpy as np
import pandas as pd


from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr

import matplotlib.pyplot as plt

import cartopy.crs as ccrs

#import test_interpolate_outputs

!pwd

/home/pxv220016/prasoon/data/SAMI3-GITM-python


In [ ]:
# !chmod -R 770 test


In [4]:
import pandas as pd

allsatdf = pd.read_csv('/petastore/phil/Prasoon_satellite_project/satfiles/20130317/DMSP_all-with_datetime.csv')

firstsat = allsatdf.loc[allsatdf.sat == 16].reset_index(drop=True)
# make sure to reset index & drop bad values, otherwise it's harder
firstsat


,glat,glon,time,sat,dayofyear,dtime
0,2.8,261.2,0.0,16,75,2013-03-16 00:00:00
1,6.3,260.5,60.0,16,75,2013-03-16 00:01:00
2,9.8,259.7,120.0,16,75,2013-03-16 00:02:00
3,13.3,258.9,180.0,16,75,2013-03-16 00:03:00
4,16.8,258.0,240.0,16,75,2013-03-16 00:04:00
...,...,...,...,...,...,...
4315,56.7,96.1,86100.0,16,77,2013-03-18 23:55:00
4316,53.3,94.3,86160.0,16,77,2013-03-18 23:56:00
4317,49.9,92.7,86220.0,16,77,2013-03-18 23:57:00
4318,46.5,91.2,86280.0,16,77,2013-03-18 23:58:00


In [5]:
latlon = [firstsat.glat, firstsat.glon]
latlon = np.array(latlon)

# no alt in files, just assume it's 840 km. Add that to latlon object
alt = np.array([840]) #[840 for i in range(len(latlon[0]))] # thus everywhere the 840km is set as the altitude

lat = latlon[0]
lon = latlon[1]


# add in alt.
# Theres's a better way to do this but I don't remember...
latlonalt = ([lat[:], lon[:], alt])

# latlonalt

In [7]:
import interpolate_outputs

# out_path = '/home/pxv220016/prasoon/data/SAMI3-GITM-python/test'
out_path = '/petastore/phil/Prasoon_satellite_project/'

sami_data_path = '/petastore/phil/Prasoon_satellite_project/more_runs/20130317/sami-gitm-coupled/'

ds = interpolate_outputs.do_interpolations(sami_data_path=sami_data_path,
                                     dtime_sim_start='20130316', #! CHANGED THIS
                                     out_lat_lon_alt=latlonalt,
                                     out_path=out_path,
                                     out_runname='-at-sat_16',
                                     save_delauney=True,
                                     max_alt=1500, #to make running faster. Not necessary.
                                     cols='edens',
                                    show_progress=True,
                                    engine='h5netcdf', #change to None if this makes errors
                                      return_ds_too=True
                                     )

attempting to reuse existing triangulation file


Reading in SAMI data:   0%|          | 0/596 [00:00<?, ?it/s]

  0%|          | 0/597 [00:00<?, ?it/s]

In [ ]:
ds.edens.isel(lon=300, lat=300).plot(x='time')

In [ ]:
#### BELOW IS JUST TO TEST THAT ds WAS WRITTEN CORRECTLY. return_ds_too ABOVE ALREADY HAS THE DATASET STORED IN MEMORY!

In [9]:
# then read the created file with:

!cd test
!pwd

import xarray as xr
import os

ds = xr.open_dataset(os.path.join(out_path, 'SAMI_REGRID-at-sat_16.nc'),
                   engine='h5netcdf') #set engine to the same one used to calculate interpolation.

print(ds)

/home/pxv220016/prasoon/data/SAMI3-GITM-python
<xarray.Dataset>
Dimensions:  (time: 597, lat: 4320, lon: 4320, alt: 1)
Coordinates:
  * time     (time) datetime64[ns] 2011-03-16 ... 2011-03-18T01:39:54
  * lat      (lat) float64 2.8 6.3 9.8 13.3 16.8 ... 56.7 53.3 49.9 46.5 43.0
  * lon      (lon) float64 261.2 260.5 259.7 258.9 258.0 ... 94.3 92.7 91.2 89.9
  * alt      (alt) int64 840
Data variables:
    edens    (time, lat, lon, alt) float64 ...
